"user_id" is unique by user
"class": whether the activity is fradulent(1) or not(0)

#Loading Packages

In [ ]:
import pandas as pd 
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
#!pip install -U imbalanced-learn   
# !pip install pandas-profiling

#Load Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ipToCountry = pd.read_csv('/content/drive/My Drive/IpAddress_to_Country.csv')
fraud = pd.read_csv('/content/drive/My Drive//imbalancedFraudDF.csv')

fraud.head() #show top 5 rows 

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0


#Exploration Data Analysis


Missing value& Duplicate value?

In [ ]:
# count of NaN in each column
fraud.isna().sum() # .isna to look for NaN value(missing value) 

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

In [ ]:
#加入查duplicate 
fraud.duplicated().sum()

0

In [ ]:
#Distribution of the label column 
fraud['class'].value_counts() 

0    136961
1      1415
Name: class, dtype: int64

In [ ]:
#Identify country info based on ip_address
ipToCountry.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [ ]:
ipToCountry['lower_bound_ip_address'] <= 16777217

0          True
1         False
2         False
3         False
4         False
          ...  
138841    False
138842    False
138843    False
138844    False
138845    False
Name: lower_bound_ip_address, Length: 138846, dtype: bool

In [ ]:
ipToCountry['upper_bound_ip_address'] >= 16777217

0         True
1         True
2         True
3         True
4         True
          ... 
138841    True
138842    True
138843    True
138844    True
138845    True
Name: upper_bound_ip_address, Length: 138846, dtype: bool

In [ ]:
ipToCountry[(ipToCountry['upper_bound_ip_address'] >= 16777217) & (ipToCountry['lower_bound_ip_address'] <= 16777217)]

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia


In [ ]:
(ipToCountry['upper_bound_ip_address'] >= 16777217) & (ipToCountry['lower_bound_ip_address'] <= 16777217)

0          True
1         False
2         False
3         False
4         False
          ...  
138841    False
138842    False
138843    False
138844    False
138845    False
Length: 138846, dtype: bool

In [ ]:

start = time.time()

countries = []  #define an empty list named"countries" 
for i in range(len(fraud)):
    #select "ip_address" feature value of the ith row from "fraud_data"
    ip_address = fraud.loc[i, 'ip_address']#number   
    #below [] is list of T/F, tmp is a df of shape n * 3, where n is 1 if found a match or 0 if no match
    tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
    if len(tmp) == 1:#found match  #len: number of rows for dataset 
    #append "country" to countries 
        countries.append(tmp['country'].values[0])
    else:#no match
        countries.append('NA')
        
fraud['country'] = countries #fill in the "country" column with value in "countries"
runtime = time.time() - start 

print("Lookup took", runtime, "seconds.")


Lookup took 193.75121426582336 seconds.


In [ ]:
ip_address = fraud.loc[6, 'ip_address']
tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
print(tmp)

       lower_bound_ip_address  upper_bound_ip_address        country
28203            1.686110e+09              1694498815  United States


Feature Engineering

In [ ]:
#as long as the numbers are the same, there's no dup
print(fraud.user_id.nunique())#138376
print(len(fraud.index))#138376
#all of the user_id has only the first 1 transaction, no way to do time based aggregates, 
#e.g. amount/counts in past 1 day for this user

138376
138376


In [ ]:
fraud

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
138371,185619,2015-01-09 02:25:03,2015-01-09 02:25:04,30,AVOVPRJNLOLWD,SEO,IE,M,35,8.031690e+08,1,Canada
138372,144031,2015-07-07 15:29:10,2015-08-20 01:48:50,67,YLHYGZVPCLHLH,Ads,FireFox,M,23,3.572177e+09,1,Israel
138373,58899,2015-04-24 15:28:22,2015-07-16 01:53:25,54,ORSHWNJVUGJFE,Direct,Safari,M,23,1.374693e+08,1,United States
138374,120669,2015-04-14 13:53:32,2015-05-18 23:52:34,31,HNQOJFPCMBRQU,SEO,Chrome,M,31,7.087370e+08,1,China


In [ ]:
#time related features: can be done before split, as they has no interaction between other rows, solely based on other columns of the same row
fraud['interval_after_signup'] = (pd.to_datetime(fraud['purchase_time']) - pd.to_datetime(fraud['signup_time'])).dt.total_seconds()

fraud['signup_days_of_year'] = pd.DatetimeIndex(fraud['signup_time']).dayofyear

#bed time operation
fraud['signup_seconds_of_day'] = pd.DatetimeIndex(fraud['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud['signup_time']).hour

fraud['purchase_days_of_year'] = pd.DatetimeIndex(fraud['purchase_time']).dayofyear
fraud['purchase_seconds_of_day'] = pd.DatetimeIndex(fraud['purchase_time']).second + 60 * pd.DatetimeIndex(fraud['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud['purchase_time']).hour

fraud = fraud.drop(['user_id','signup_time','purchase_time'], axis=1)

In [ ]:
fraud.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
0,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,55,82549,108,10031
1,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,158,74390,159,5934
2,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,492085.0,118,76405,124,50090
3,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,202,25792,252,67253
4,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,Canada,4240931.0,141,21783,190,29114


In [ ]:
print(fraud.source.value_counts())

SEO       55766
Ads       54913
Direct    27697
Name: source, dtype: int64


Split Train/ Testing dataset


In [ ]:
y = fraud['class']
X = fraud.drop(['class'], axis=1) #axis=1 drop from column; axis = 0 drop from row

#split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) #train_test_split function
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

X_train.shape: (110700, 13)
y_train.shape: (110700,)


In [ ]:
X_train['country'].value_counts(ascending=True)

San Marino            1
Madagascar            1
Benin                 1
Bermuda               1
Fiji                  1
                  ...  
United Kingdom     3253
Japan              5251
China              8876
NA                16275
United States     42348
Name: country, Length: 177, dtype: int64

In [ ]:
X_train.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
29343,12,OULPAZAFRFPXP,Ads,Chrome,M,42,3.690922e+09,Korea Republic of,3499664.0,183,67384,224,24648
12190,10,AIIWMFEYQQIEB,Ads,Opera,M,29,1.686759e+09,United States,6766039.0,5,78146,84,18585
19388,34,VUVETBUPCIWJE,Direct,Chrome,M,53,4.138429e+09,NA,5870515.0,197,81354,265,76669
89104,48,QCFULAJOYKFUU,Ads,Chrome,M,29,9.617337e+07,France,2145618.0,160,30920,185,16538
82082,44,IHRWLMIJMEEEU,Ads,FireFox,M,24,1.936025e+09,China,7079059.0,111,71897,193,66156


Convert categorical features with high cadinality to numericals

In [ ]:
#converting needs to be done after split
X_train = pd.get_dummies(X_train, columns=['source', 'browser'])#need to drop ['source', 'browser']? no, auto dropped by get_dummies 
X_train['sex'] = (X_train.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_train_device_id_mapping = X_train.device_id.value_counts(dropna=False) #dataset.feature.function
X_train['n_dev_shared'] = X_train.device_id.map(X_train_device_id_mapping)# number of times device_id occurred in train data
#use"X_train_device_id_mapping"to replace device.id

# the more a ip is shared, the more suspicious
X_train_ip_address_mapping = X_train.ip_address.value_counts(dropna=False)
X_train['n_ip_shared'] = X_train.ip_address.map(X_train_ip_address_mapping)

# the less visit from a country, the more suspicious
X_train_country_mapping = X_train.country.value_counts(dropna=False)
X_train['n_country_shared'] = X_train.country.map(X_train_country_mapping)#lots of NAs in country column, #without dropna=False will produce nan in this col


X_train = X_train.drop(['device_id','ip_address','country'], axis=1)

In [ ]:
a = fraud.device_id.value_counts(dropna=False)

In [ ]:
fraud.device_id.map(a)

0         1
1         1
2         1
3         1
4         1
         ..
138371    3
138372    1
138373    1
138374    1
138375    1
Name: device_id, Length: 138376, dtype: int64

get_dummies can be used to do one hot encoding

In [ ]:
X_test = pd.get_dummies(X_test, columns=['source', 'browser'])
X_test['sex'] = (X_test.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)

In [ ]:
X_train.head()

,purchase_value,sex,age,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,n_dev_shared,n_ip_shared,n_country_shared
29343,12,1,42,3499664.0,183,67384,224,24648,1,0,0,1,0,0,0,0,1,1,3075
12190,10,1,29,6766039.0,5,78146,84,18585,1,0,0,0,0,0,1,0,1,1,42348
19388,34,1,53,5870515.0,197,81354,265,76669,0,1,0,1,0,0,0,0,1,1,16275
89104,48,1,29,2145618.0,160,30920,185,16538,1,0,0,1,0,0,0,0,1,1,2322
82082,44,1,24,7079059.0,111,71897,193,66156,1,0,0,0,1,0,0,0,1,1,8876


In [ ]:
X_train.n_dev_shared.value_counts(dropna=False)

1    105427
2      4774
3       324
4       124
5        45
6         6
Name: n_dev_shared, dtype: int64

In [ ]:
X_test.n_dev_shared.value_counts(dropna=False)

1    27330
2      334
3       12
Name: n_dev_shared, dtype: int64

# Modeling

LogisticRegression model

In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

# predict on test
y_pred=logreg.predict(X_test)

In [ ]:
# The model prefer to predict all obs to 0. Not a good model
cm = metrics.confusion_matrix(y_test, y_pred)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)

        pred_0  pred_1
true_0   27389       0
true_1     287       0


Random Forest

In [ ]:
# RF model could identify about half of fruad obs.
classifier_RF = RandomForestClassifier(random_state=0)

classifier_RF.fit(X_train, y_train)

# predict class labels 0/1 for the test set
predicted = classifier_RF.predict(X_test)

# generate class probabilities
probs = classifier_RF.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0

accuracy_score is: : 0.9948692007515537
roc_auc_score is: : 0.7801672204169557
f1_score is: : 0.6712962962962962
confusion_matrix is: 
        pred_0  pred_1
true_0   27389       0
true_1     142     145
recall = 0.5052264808362369
precision = 1.0


Support Vector Machine

In [ ]:
from sklearn import svm
#Create a svm Classifier
svmclassifier = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
svmclassifier.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = svmclassifier.predict(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0

XGBoost

In [ ]:
# !pip install xgboost 
import xgboost as xgb
from sklearn.metrics import mean_squared_error


In [ ]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

[19:01:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, preds)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, preds)))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, preds )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, preds)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0

accuracy_score is: : 0.9899551958375488
roc_auc_score is: : 0.5156794425087108
f1_score is: : 0.060810810810810814
confusion_matrix is: 
        pred_0  pred_1
true_0   27389       0
true_1     278       9
recall = 0.0313588850174216
precision = 1.0


SMOTE sampling

In [ ]:
#SMOTE should be appiled in train data set, and validates in test data set

#sampling_strategy = number of samples in the majority class over the number of samples in the minority class after resampling

smote = SMOTE(random_state=12)
x_train_sm, y_train_sm = smote.fit_sample(X_train, y_train)

unique, counts = np.unique(y_train_sm, return_counts=True)

print(np.asarray((unique, counts)).T)

[[     0 109572]
 [     1 109572]]


In [ ]:
#RF on smoted training data
classifier_RF_sm = RandomForestClassifier(random_state=0)

classifier_RF_sm.fit(x_train_sm, y_train_sm)

# predict class labels for the test set
predicted_sm = classifier_RF_sm.predict(X_test)

# generate class probabilities
probs_sm = classifier_RF_sm.predict_proba(X_test)


# generate evaluation metrics
print("%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
print("%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
print("%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm )))#string to int

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall or sens_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1]))
print('precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1]))

#different AUC??

accuracy_score_sm is: : 0.9948692007515537
roc_auc_score_sm is: : 0.7674601810564352
f1_score_sm is: : 0.6712962962962962
confusion_matrix_sm is: 
        pred_0  pred_1
true_0   27389       0
true_1     142     145
recall or sens_sm = 0.5052264808362369
precision_sm = 1.0


XgBoost SMOTE

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(x_train_sm,y_train_sm)
preds = xg_reg.predict(X_test)

[19:14:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, preds)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, preds)))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, preds )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, preds)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0

accuracy_score is: : 0.9934961699667582
roc_auc_score is: : 0.7519195312648087
f1_score is: : 0.6170212765957447
confusion_matrix is: 
        pred_0  pred_1
true_0   27351      38
true_1     142     145
recall = 0.5052264808362369
precision = 0.7923497267759563


# Parameter tuning by GridSearchCV

In [ ]:
# Eval metrics to be calculated for each combination of parameters and cv, they are all calculated(by scoring), but not necessarily used to pick the optimal(by refit)
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score, pos_label=1)
    
}

In [ ]:
def grid_search_wrapper(model, parameters, refit_score='f1_score'): 
    """
    fits a GridSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """

    grid_search = GridSearchCV(model, parameters, scoring=scorers, refit=refit_score, #model and parameter defined by previous function
                           cv=3, return_train_score=True, n_jobs=-1)
    
    grid_search.fit(x_train_sm, y_train_sm)

    # make the predictions
    y_pred = grid_search.predict(X_test)
    y_prob = grid_search.predict_proba(X_test)[:, 1]
    
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    cm = confusion_matrix(y_test, y_pred)
    cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
    print(cmDF)
    
    print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, y_pred)))#string to int

    print('recall = ', float(cm[1,1]) / (cm[1,0] + cm[1,1]))
    print('precision = ', float(cm[1,1]) / (cm[1, 1] + cm[0,1]))

    return grid_search

Optimizing on f1_score on LR

In [ ]:
LRGrid = {"C" : np.logspace(-2,2,5), "penalty":["l1","l2"]}# l1 lasso l2 ridge
#param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
logRegModel = LogisticRegression(random_state=0)

grid_search_LR_f1 = grid_search_wrapper(logRegModel, LRGrid, refit_score='f1_score')

Best params for f1_score
{'C': 0.01, 'penalty': 'l2'}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   19611    7778
true_1     101     186
	roc_auc_score is: : 0.6935124772871634
	f1_score is: : 0.0450854441885832
recall =  0.6480836236933798
precision =  0.0233550979407333


Optimizing on f1_score on RF

In [ ]:
parameters = {        
'max_depth': [None, 5, 15],
'n_estimators' :  [10,150],
'class_weight' : [{0: 1, 1: w} for w in [0.2, 1, 100]]
}

clf = RandomForestClassifier(random_state=0)

In [ ]:
grid_search_rf_f1 = grid_search_wrapper(clf, parameters, refit_score='f1_score') 

Best params for f1_score
{'class_weight': {0: 1, 1: 0.2}, 'max_depth': None, 'n_estimators': 10}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27389       0
true_1     142     145
	roc_auc_score is: : 0.7537836281332202
	f1_score is: : 0.6712962962962962
recall =  0.5052264808362369
precision =  1.0


In [ ]:
best_rf_model_f1 = grid_search_rf_f1.best_estimator_
best_rf_model_f1

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 0.2}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [ ]:
results_f1 = pd.DataFrame(grid_search_rf_f1.cv_results_)
results_sortf1 = results_f1.sort_values(by='mean_test_f1_score', ascending=False)
results_sortf1[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()

,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
0,1.0,0.527,0.69,1.0,0.857,0.923,None,"{0: 1, 1: 0.2}",10
7,1.0,0.527,0.69,1.0,1.000,1.000,None,"{0: 1, 1: 1}",150
13,1.0,0.527,0.69,1.0,1.000,1.000,None,"{0: 1, 1: 100}",150
11,1.0,0.527,0.69,1.0,0.584,0.737,15,"{0: 1, 1: 1}",150
1,1.0,0.527,0.69,1.0,1.000,1.000,None,"{0: 1, 1: 0.2}",150


Feature Importance

In [ ]:
pd.DataFrame(best_rf_model_f1.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)

,importance
interval_after_signup,0.417489
purchase_days_of_year,0.125334
purchase_seconds_of_day,0.081799
signup_seconds_of_day,0.078478
n_dev_shared,0.072574
signup_days_of_year,0.054064
purchase_value,0.043650
age,0.039094
n_country_shared,0.024125
n_ip_shared,0.017022


Optimizing recall_score on RF

In [ ]:
grid_search_rf_recall = grid_search_wrapper(clf, parameters, refit_score='recall_score')

Best params for recall_score
{'class_weight': {0: 1, 1: 100}, 'max_depth': 5, 'n_estimators': 150}

Confusion matrix of Random Forest optimized for recall_score on the test data:
        pred_0  pred_1
true_0   27146     243
true_1     132     155
	roc_auc_score is: : 0.7904661234456265
	f1_score is: : 0.4525547445255475
recall =  0.5400696864111498
precision =  0.38944723618090454


In [ ]:
best_RF_model_recall = grid_search_rf_recall.best_estimator_
best_RF_model_recall

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 100}, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=150, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [ ]:
# predict class labels for the test set
predictedBest_recall = best_RF_model_recall.predict(X_test)

# generate class probabilities
probsBest_recall = best_RF_model_recall.predict_proba(X_test)

results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)# recall score is different from above, as above is metric on test data, this is performance on cv data
results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()
#recall is worse than default rf?? no this is on test, but train recall is better

,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
15,0.159,0.636,0.254,0.164,0.656,0.262,5,"{0: 1, 1: 100}",150
14,0.160,0.633,0.255,0.162,0.652,0.260,5,"{0: 1, 1: 100}",10
16,0.675,0.533,0.593,0.759,0.813,0.782,15,"{0: 1, 1: 100}",10
0,1.000,0.527,0.690,1.000,0.857,0.923,None,"{0: 1, 1: 0.2}",10
1,1.000,0.527,0.690,1.000,1.000,1.000,None,"{0: 1, 1: 0.2}",150


Parameter tuning by Random
SearchCV

In [ ]:
def random_search_wrapper(model, parameters, refit_score='f1_score'): 
    """
    fits a RandomSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """

    random_search = RandomizedSearchCV(model, parameters, scoring=scorers, refit=refit_score, #model and parameter defined by previous function
                           cv=3, return_train_score=True, n_jobs=-1)
    
    random_search.fit(x_train_sm, y_train_sm)

    # make the predictions
    y_pred = random_search.predict(X_test)
    y_prob = random_search.predict_proba(X_test)[:, 1]
    
    print('Best params for {}'.format(refit_score))
    print(random_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    cm = confusion_matrix(y_test, y_pred)
    cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
    print(cmDF)
    
    print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, y_pred)))#string to int

    print('recall = ', float(cm[1,1]) / (cm[1,0] + cm[1,1]))
    print('precision = ', float(cm[1,1]) / (cm[1, 1] + cm[0,1]))

    return random_search

Optimizing on f1_score on LR

In [ ]:
parameters_LR = {"C" : np.logspace(-2,2,5), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logRegModel = LogisticRegression(random_state=0)

Random_search_LR_f1 = random_search_wrapper(logRegModel, parameters_LR, refit_score='f1_score')

Best params for f1_score
{'penalty': 'l2', 'C': 0.01}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   19611    7778
true_1     101     186
	roc_auc_score is: : 0.6935124772871634
	f1_score is: : 0.0450854441885832
recall =  0.6480836236933798
precision =  0.0233550979407333


Optimizing on f1_score on RF

In [ ]:
parameters_rf = {        
'max_depth': [None, 5, 15],
'n_estimators' :  [10,150],
'class_weight' : [{0: 1, 1: w} for w in [0.2, 1, 100]]
}

clf = RandomForestClassifier(random_state=0)
Random_search_RF_f1 = random_search_wrapper(clf, parameters, refit_score='f1_score')

Best params for f1_score
{'n_estimators': 150, 'max_depth': None, 'class_weight': {0: 1, 1: 0.2}}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27389       0
true_1     142     145
	roc_auc_score is: : 0.7595866521352006
	f1_score is: : 0.6712962962962962
recall =  0.5052264808362369
precision =  1.0


In [ ]:
best_rf_model_f1 = Random_search_RF_f1.best_estimator_
best_rf_model_f1

Optimizing on f1_score on XgBoost

In [ ]:
parameters_xg = {        


}
xg = xgb.XGBClassifier()
Random_search_xg_f1 = random_search_wrapper(xg, parameters, refit_score='f1_score')

Optimizing recall_score on RF

Optimizing recall_score on XgBoost